# Laborator II

## Analiza sintactica

In laboratorul trecut am vazut cum putem extrage tokenii dintr-un text. Folosind doar analiza lexicala nu putem verifica daca tokenii respecta o anumita ordine, ci doar daca exista. Pentru a verifica daca avem si o ordine corecta a tokenilor trebuie sa construim o gramatica care verifica daca un anumit text ar fi putut fi generat de ea.

Exemplu: Se dau mai multe liste de numere de forma [1, 2, 3]; [4, 5, 6]; ... ; [7, 8, 9, 10]. Calculati suma lor.

In primul rand trebuie sa gasim tokenii: ([, ], ',' , digit, spatiu*, ;)

In [160]:
from sly import Lexer, Parser

In [161]:
class ListsLexer(Lexer):
    tokens = { OPEN_BRACE, CLOSE_BRACE, COMMA, DIGITS, SEMICOLON }
    ignore = ' \t'             

    OPEN_BRACE = r'\['
    CLOSE_BRACE = r'\]'
    COMMA = ','
    DIGITS = r'\d+'
    SEMICOLON = r';'
    
    ignore_newline = r'\n+'

    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1

In [162]:
input_text = ' [1, 2, 3]; [4, 5, 6]; [7, 8, 9, 10];'
lexer = ListsLexer()

for token in lexer.tokenize(input_text):
    print(token)

Token(type='OPEN_BRACE', value='[', lineno=1, index=1)
Token(type='DIGITS', value='1', lineno=1, index=2)
Token(type='COMMA', value=',', lineno=1, index=3)
Token(type='DIGITS', value='2', lineno=1, index=5)
Token(type='COMMA', value=',', lineno=1, index=6)
Token(type='DIGITS', value='3', lineno=1, index=8)
Token(type='CLOSE_BRACE', value=']', lineno=1, index=9)
Token(type='SEMICOLON', value=';', lineno=1, index=10)
Token(type='OPEN_BRACE', value='[', lineno=1, index=12)
Token(type='DIGITS', value='4', lineno=1, index=13)
Token(type='COMMA', value=',', lineno=1, index=14)
Token(type='DIGITS', value='5', lineno=1, index=16)
Token(type='COMMA', value=',', lineno=1, index=17)
Token(type='DIGITS', value='6', lineno=1, index=19)
Token(type='CLOSE_BRACE', value=']', lineno=1, index=20)
Token(type='SEMICOLON', value=';', lineno=1, index=21)
Token(type='OPEN_BRACE', value='[', lineno=1, index=23)
Token(type='DIGITS', value='7', lineno=1, index=24)
Token(type='COMMA', value=',', lineno=1, index=

### Parser

Observam ca putem extrage tokenii, dar se poate sa nu fie in ordinea corecta (de ex: [[,123,]1[]). Implementam parser-ul.

Pentru parser trebuie mai intai sa gasim o gramatica care recunoaste liste scrise corect:

<ul>
    <li>lists -> list | list lists</li>
    <li>list -> OPEN_BRACE numbers CLOSE_BRACE SEMICOLON</li>
    <li>numbers -> DIGITS | numbers COMMA DIGITS</li>
</ul>

Explicatie:
<ul>
    <li>Un input corect (mai multe liste) este compus fie dintr-o singura lista, fie este o lista urmata de alte liste</li>
    <li>O lista incepe cu [, dupa care urmeaza mai multe numere, apoi trebuie sa apara ] si punct si virgula (;)</li>
    <li>Numerele pot fi reprezentate de un singur numar (DIGITS) sau sa apara mai multe numere (numbers) urmate de virgula si de un numar</li>
</ul>


Implementam parser-ul scriind pentru fiecare productie (A -> BC) cate o regula si ce trebuie sa faca parser-ul cand identifica regula.

Pentru fiecare regula scriem cate o functie care este decorata \(@_\) cu  partea dreapta a productiei. Numele functiei este dat de partea stanga a productiei. De exemplu pentru productia "numbers -> DIGITS" avem urmatoarea definitie de functie:

<block>
@_('DIGITS')<br>
def numbers(self, p):<br>
</block>

Parametrul p (productie) al functiei numbers ne permite sa accesam partea dreapta a productiei (in cazul de fata avem acces la p.DIGITS care reprezinta pe rand numere (1, 4 si 7; de ce nu si restul numerelor?)

In [163]:
class ListsParser(Parser):
    tokens = ListsLexer.tokens
        
    # lists -> list
    @_('list')
    def lists(self, p):
        return [p.list]
    
    # lists -> list lists
    @_('list lists')
    def lists(self, p):
        current_list = p.list
        rest_of_lists = list(p.lists)
        
        all_lists = [current_list] + rest_of_lists
            
        return all_lists
    
    # list -> OPEN_BRACE numbers CLOSE_BRACE SEMICOLON
    @_('OPEN_BRACE numbers CLOSE_BRACE SEMICOLON')
    def list(self, p):
        return p.numbers
    
    # numbers -> DIGITS
    @_('DIGITS')
    def numbers(self, p):
        return [int(p.DIGITS)]

    # numbers -> numbers COMMA DIGITS
    @_('numbers COMMA DIGITS')
    def numbers(self, p):
        current_number = int(p.DIGITS)
        current_list = list(p.numbers)
        
        current_list.append(current_number)
        return current_list

In [164]:
input_text = ' [1, 2, 3]; [4, 5, 6]; [7, 8, 9, 10];'

lexer = ListsLexer()
parser = ListsParser()

lists = parser.parse(lexer.tokenize(input_text))
print(sum(map(sum, lists)))

55


### Exercitiu.

Modificati exemplul anterior astfel incat sa returnati direct suma numerelor, fara a mai salva listele intermediare. Hint: puteti defini un constructor pentru clasa ListsParser si sa folositi variabile de instanta (de exemplu self.sum = 0).

In [168]:
# Cod

### Exercitiu

Modificati exemplul anterior astfel incat sa efectuati suma fiecarei liste si apoi produsul sumelor. Exemplu pentru: [1, 2, 3]; [4, 5, 6]; [7, 8, 9, 10]; sumele sunt 6, 15 si 34, iar produsul este 3060.

In [169]:
# Cod

### Exercitiu.

Primind un cod de tipul foreach(int x : v[1, n]) translatati-l in codul echivalentul pentru C (atentie la x, v, 1, n). Rezultatul ar trebui sa fie un string.

In [167]:
class SyntacticSugarLexer(Lexer):
    tokens = { TOKENS }
    ignore = ' \t'             

    # Modify TOKENS
    TOKENS = r'.'
    
    ignore_newline = r'\n+'

    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1
        
class SyntacticSugarParser(Parser):
    tokens = SyntacticSugarLexer.tokens
        
    @_('TOKENS')
    def rule(self, p):
        return p.TOKENS
    
input_text = 'foreach(int x : v[1, n])'

''' Lexer test '''
lexer = SyntacticSugarLexer()
for token in lexer.tokenize(input_text):
    print(token)

''' Parser test '''
lexer = SyntacticSugarLexer()
parser = SyntacticSugarParser()

print(parser.parse(lexer.tokenize(input_text)))

Token(type='TOKENS', value='f', lineno=1, index=0)
Token(type='TOKENS', value='o', lineno=1, index=1)
Token(type='TOKENS', value='r', lineno=1, index=2)
Token(type='TOKENS', value='e', lineno=1, index=3)
Token(type='TOKENS', value='a', lineno=1, index=4)
Token(type='TOKENS', value='c', lineno=1, index=5)
Token(type='TOKENS', value='h', lineno=1, index=6)
Token(type='TOKENS', value='(', lineno=1, index=7)
Token(type='TOKENS', value='i', lineno=1, index=8)
Token(type='TOKENS', value='n', lineno=1, index=9)
Token(type='TOKENS', value='t', lineno=1, index=10)
Token(type='TOKENS', value='x', lineno=1, index=12)
Token(type='TOKENS', value=':', lineno=1, index=14)
Token(type='TOKENS', value='v', lineno=1, index=16)
Token(type='TOKENS', value='[', lineno=1, index=17)
Token(type='TOKENS', value='1', lineno=1, index=18)
Token(type='TOKENS', value=',', lineno=1, index=19)
Token(type='TOKENS', value='n', lineno=1, index=21)
Token(type='TOKENS', value=']', lineno=1, index=22)
Token(type='TOKENS', v

yacc: Syntax error at line 1, token=TOKENS


### Exercitiu

Construiti un **parser** care verifica daca primeste cuvinte de tipul a^n b^n cu n >= 1. Exemplu: aabbb => NU, aaabbb => DA, ab => DA.

In [181]:
class AnBnLexer(Lexer):
    tokens = { ''' TODO ''' }
    ignore = ' \t'             

    ''' TODO '''
    
    ignore_newline = r'\n+'

    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1
        
class AnBnParser(Parser):
    tokens = AnBnLexer.tokens
        
    ''' TODO '''
    
    
input_text = 'aaabbbb'

''' Lexer test '''
lexer = AnBnLexer()
for token in lexer.tokenize(input_text):
    print(token)

''' Parser test '''
lexer = AnBnLexer()
parser = AnBnParser()

print(parser.parse(lexer.tokenize(input_text)))

ERROR: no grammar rules are defined


YaccError: Invalid grammar

### Exercitiu

Construiti un parser care sa returneze o lista de valori de tip bool, care reprezinta daca un cuvant este palindrom sau nu. Pentru input-ul "abba aca accas a", parser-ul trebuie sa returneze [True, True, False, True].

In [182]:
class PalindromicLexer(Lexer):
    tokens = { ''' TODO ''' }
    ignore = ' \t'             

    ''' TODO '''
    
    ignore_newline = r'\n+'

    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1
        
class PalindromicParser(Parser):
    tokens = AnBnLexer.tokens
        
    ''' TODO '''
    
    
input_text = 'abba aca accas a'

''' Lexer test '''
lexer = PalindromicLexer()
for token in lexer.tokenize(input_text):
    print(token)

''' Parser test '''
lexer = PalindromicLexer()
parser = PalindromicParser()

print(parser.parse(lexer.tokenize(input_text)))

ERROR: no grammar rules are defined


YaccError: Invalid grammar

### Exercitiu

Construiti un parser care primeste o expresie in forma infix si returneaza expresia in forma postfixata. 

Exemplu pentru input-ul: (a+b)\*c, parser-ul returneaza ab+c*

In [183]:
class InfixToPostfixLexer(Lexer):
    tokens = { ''' TODO ''' }
    ignore = ' \t'             

    ''' TODO '''
    
    ignore_newline = r'\n+'

    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1
        
class InfixToPostfixParser(Parser):
    tokens = AnBnLexer.tokens
        
    ''' TODO '''
    
    
input_text = 'abba aca accas a'

''' Lexer test '''
lexer = InfixToPostfixLexer()
for token in lexer.tokenize(input_text):
    print(token)

''' Parser test '''
lexer = InfixToPostfixLexer()
parser = InfixToPostfixParser()

print(parser.parse(lexer.tokenize(input_text)))

ERROR: no grammar rules are defined


YaccError: Invalid grammar